In [ ]:
from ocpy import OCP, DDPSolver, iLQRSolver
from ocpy import symutils

import numpy as np
import sympy as sym
from sympy import sin, cos, tan, exp, log, ln, sinh, cosh, tanh, diff, sqrt
from IPython.display import display, Math


In [ ]:
# Dimensions of state and input
n_x = 4
n_u = 1
sim_name = 'cartpole'
# Define ocp class
ocp = OCP(n_x, n_u, sim_name)
# Get symbols
t = ocp.get_t()
x = ocp.get_x()
u = ocp.get_u()


In [ ]:
# Symbolic expressions of constants.
m_c, m_p, l, g, u_min, u_max, u_eps = ocp.define_scalar_constants(
         [('m_c', 2), ('m_p', 0.1), ('l', 0.5), ('g', 9.80665), 
          ('u_min', -15),  ('u_max', 15), ('u_eps', 0.001)]
          )
# Cost weight
q = ocp.define_vector_constant('q', [2.5, 10, 0.01, 0.01])
r = ocp.define_vector_constant('r', [1])
q_f = ocp.define_vector_constant('q_{f}', [2.5, 10, 0.01, 0.01])
Q = symutils.diag(q)
Q_f = symutils.diag(q_f)
R = symutils.diag(r)
# Reference state. 
x_ref = ocp.define_vector_constant('x_{ref}', [0, np.pi, 0, 0])


In [ ]:
# State of equation.
f = ocp.get_zero_vector(n_x)
f[0] = x[2]
f[1] = x[3]
f[2] = (u[0] + m_p*sin(x[1])*(l*x[1]*x[1] + g*cos(x[1])) )/( m_c+m_p*sin(x[1])*sin(x[1]) )
f[3] = (-u[0] * cos(x[1]) - m_p*l*x[1]*x[1]*cos(x[1])*sin(x[1]) 
        - (m_c+m_p)*g*sin(x[1]) )/( l*(m_c + m_p*sin(x[1])*sin(x[1])))
# Log barrier for control limits.
u_barrier = sym.Matrix([sum(-ln(u[i] - u_min) - ln(u_max - u[i]) for i in range(n_u))*1e-5])
# Stage cost and terminal cost.
l = (x - x_ref).T * Q * (x - x_ref) + u.T * R * u + u_barrier
lf = (x - x_ref).T * Q_f * (x - x_ref)
# Display state equation and cost function
display(Math(r"\dot{x} = f(x, u, t) \equiv %s" % sym.latex(f)))
display(Math(r"l(x, u) = %s" % sym.latex(l)))
display(Math(r"l_f(x) = %s" % sym.latex(lf)))


In [ ]:
# Horizon length and discretization grids.
T = 5.0
N = 200
# Initial condition
t0 = 0.0
x0 = np.array([0.0, 0.0, 0.0, 0.0])
# Define ocp
ocp.define_unconstrained(f, l, lf, t0, x0, T, N)


In [ ]:
# Hand over ocp to solver.
# DDP. Based on exact Hessian.
solver = DDPSolver(ocp)

# iLQR. Based on Gauss-Newton-like approximated Hessian.
# solver = iLQRSolver(ocp)


In [ ]:
# if needed, set solution guess.
us_guess = np.zeros((N, n_u))
solver.set_guess(us_guess=us_guess)


In [ ]:
# list of step size for line search.
alphas = [0.5**i for i in range(8)]

# set parameters.
solver.set_alphas(alphas)
solver.set_regularization_coeff(
    gamma_init=1e-3, rho_gamma=5.0, gamma_min=0.0, gamma_max=1e6
)
solver.set_stop_tol(1e-3)
solver.set_max_iters(1000)

# Solve ocp
xs, us, ts, is_success = solver.solve(result=True, log=True, plot=True, gamma_fixed=0.0)


In [ ]:
%matplotlib inline
# Visualize
from ocpy.animator import CartPoleAnimator
animator = CartPoleAnimator(solver.get_log_directory(), sim_name)
animator.generate_animation(False)


In [ ]:
print('xs\n', xs)
print('us\n', us)


In [ ]:
result = solver.get_result()
gammas = result['gammas']

import matplotlib.pyplot as plt
# plt.yscale('log')
plt.plot(gammas)
